## Web Search and Scrape
The **Web Search and Scrape** ant demonstrates the use of a Web API secured using an API key with an auth header. 
This sample requires you to create a Web Search (bing) resource in Azure and enter its corresponding API key in [0-AI-settings](0-AI-settings.ipynb)

This demo uses a lot of tokens. If your quota is low, you will get throttled.

## First steps
- First, download AntRunLib from Nuget
- At least once, set up the environment using **[0-AI-settings](0-AI-settings.ipynb)**

In [1]:
#r "nuget: AntRunnerLib, 0.9.5"

using AntRunnerLib;
using AntRunnerLib.Identity;
using static AntRunnerLib.ClientUtility;
using System.IO;

#!import config/Settings.cs 

var envVariables = Settings.GetEnvironmentVariables();
foreach (var kvp in envVariables)
{
    Environment.SetEnvironmentVariable(kvp.Key, kvp.Value);
}

var config = AzureOpenAiConfigFactory.Get();
var client = GetOpenAiClient(config);


Installed Packages AntRunnerLib, 0.9.5

## Ensure the Web Search and Scrape assistant exists
The definition of this assistant is located in the ".\AssistantDefinitions\Web Search and Scrape" folder

".\AssistantDefinitions\" is a default path. You can override it by setting the **ASSISTANTS_BASE_FOLDER_PATH** environment variable.

"Web Search and Scrape" contains the following files:
```
│   HtmlAgilityPack.dll - Loaded by the notebook, a great open library for c# web scraping
│   HtmlAgilityPackExtensions.dll - Loaded by the notebook, part of this project to do a simple conversion to markdown for less tokens and noise
│   HtmlAgilityPackExtensions.pdb - If you want to debug
│   instructions.md - The prompt
│   manifest.json - The assistant definition
│   
└───OpenAPI
        api.bing.microsoft.com.json - Azure Web Search (crawl operation)
        auth.json - Auth setup for web search
        tool.localhost.json - Local HtmlAgility.HtmlAgilityPackExtensions.ConvertUrlToMarkdown c# function 
```

## Crawl versus ConvertUrlToMarkdown tools

Crawl returns search results with snippets and only has a small summary or excerpt of the page

ConvertUrlToMarkdown fetches the whole page and turns it all into markdown for consumption

## auth.json
AntRunnerLib uses the instructions in this file to create the correct auth header(s) for the API. In this case, a header is added to the web request, **Ocp-Apim-Subscription-Key**, the header's value is supplied by an environment variable named, SEARCH_API_KEY. The environment variable name is user defined and ensuring it has a value is the responsibility of the application. 

```json
{
  "hosts": {
    "api.bing.microsoft.com": {
      "auth_type": "service_http",
      "header_name": "Ocp-Apim-Subscription-Key",
      "header_value_env_var": "SEARCH_API_KEY"
    }
  }
}
```

### Explanation
`AssistantUtility.Create` acts based on the contents of a folder structure following a convention laid out in these notebooks. In this case, it copied the contents of instructions.md into the assistant manifest.json. It also parsed the api.bing.microsoft.com swagger which follows an Open API schema into tool defintions for the assitant.

In [2]:
var assistantId = await AssistantUtility.Create("Web Search and Scrape", config);
Console.WriteLine(assistantId)

asst_zt1qBxqen6EANfExpqhUVMoS


In [3]:
AntLoader.LoadAssembly(@".\AssistantDefinitions\Web Search and Scrape\HtmlAgilityPack.dll");
AntLoader.LoadAssembly(@".\AssistantDefinitions\Web Search and Scrape\HtmlAgilityPackExtensions.dll");

## Run the Assistant

`output.Dialog` shows the conversation.

In [4]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "Web Search and Scrape",
    Instructions = "Use python to get todays date and then use it to find things to do in Atlanta today. Be careful to not include events in the past in your final answer.",
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output.Dialog.DisplayAs("text/markdown")


User: Use python to get todays date and then use it to find things to do in Atlanta today. Be careful to not include events in the past in your final answer.

Assistant: I ran this code:
```
from datetime import datetime

# Get today's date
today_date = datetime.now().strftime('%Y-%m-%d')
today_date
```

I called the tool named crawl with these arguments:
```
{"q":"things to do in Atlanta December 31 2024","count":10,"textFormat":"HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta December 31 2024"},"webPages":{"value":[{"name":"Atlanta Events in December 2024 - Schedule \u0026amp; Tickets","url":"https://atlanta-ga.events/december/","displayUrl":"https://atlanta-ga.events/december","snippet":"Check Out The Atlanta Events Calendar for December 2024: Current \u0026amp; Upcoming Events \u002B Best \u0026amp; Fun Thing To Do in the City. ... Dec 31, 2024 3:30 AM . Georgia Tech Yellow Jackets Basketball . Events: 3. McCamish Pavilion . Dec 31, 2024 2:30 PM ..."},{"name":"11 Atlanta holiday events happening this weekend | 11alive.com","url":"https://www.11alive.com/article/entertainment/events/11-things-to-do-in-atlanta-dec-26-31/85-46d65687-9cd6-4513-9fcb-5e2012452a26","displayUrl":"https://www.11alive.com/article/entertainment/events/11-things-to-do-in-atlanta-dec-26...","snippet":"Updated: 7:20 AM EST December 26, 2024 ATLANTA \u2014 The holiday season is in full swing, and Atlanta is buzzing with exciting events to enjoy this weekend! ... Date and Location: Dec. 25\u201331; 10 a ...","thumbnailUrl":"https://www.bing.com/th?id=OIF.DuWLyAoWt9tU6lqzr3sKvA\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events - December 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-december.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-december.html","snippet":"Things to do in Atlanta, GA in December, including upcoming festivals, Christmas events and other holiday celebrations, lights shows, holiday markets, comedy, musicals, ballet, tours of homes, and more. ... December 31 2024 New Year\u2019s eve event at the Georgia Aquarium, with DJ and dancing, food, and after-hours access to the aquatic exhibits.","thumbnailUrl":"https://www.bing.com/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Events \u0026amp; Things To Do in Atlanta in December 2024 - AllEvents.in","url":"https://allevents.in/atlanta/december","displayUrl":"https://allevents.in/atlanta/december","snippet":"Find events and things to do in december 2024 in Atlanta. Discover parties, concerts, meets,shows, sports, club, reunion, Performance happening in 2024 in Atlanta. Open Menu. Atlanta Change City; ... Tue, 31 Dec Trending events in december in Atlanta. Atlanta Symphony Orchestra: New Year\u0026#39;s Eve"},{"name":"Events \u0026amp; Things to Do for Christmas in Atlanta - Discover Atlanta","url":"https://discoveratlanta.com/events/seasonal/christmas/","displayUrl":"https://discoveratlanta.com/events/seasonal/christmas","snippet":"The possibilities are endless with so many things to do in Atlanta at Christmas! ... Atlanta Ballet\u2019s The Nutcracker returns this December at Cobb Energy Centre with a larger-than-life production of a winter wonderland and dream-like world of ... 12/31/24. Candlelight Concerts 2024. Pets. 12/30/24 \u2013 1/1/25. Adopt Any Senior Pet for FREE in ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.t6GjYUDzSex1Myxpnsk1ngHaE6\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"December 31 events in Atlanta, GA - Eventbrite","url":"https://www.eventbrite.com/d/ga--atlanta/december-31/","displayUrl":"https://www.eventbrite.com/d/ga--atlanta/december-31","snippet":"HUGEL NEW YEARS EVE | Make The Girls Dance 2024 | District Atlanta. Tue, Dec 31, 10:00 PM. District Atlanta. Save this event: HUGEL NEW YEARS EVE ... TUESDAY DECEMBER 31 2024 Save this event: NYE at EMBR LOUNGE| 8PM-3AM| TUESDAY DECEMBER 31 2024. THE LEMON DROP - NEW YEARS EVE 2K24. Tue, Dec 31, 9:00 PM. Westside Cultural Arts Center."},{"name":"Explore Atlanta Events for Today, This Week or Weekend","url":"https://discoveratlanta.com/events/main/","displayUrl":"https://discoveratlanta.com/events","snippet":"Explore the Atlanta events calendar for things to do today, this week or this weekend. Find upcoming festivals, seasonal events and more! ... New Year\u2019s Eve Celebration at Red Phone Booth 2024 12/31/24. New Year\u2019s Eve Dinner at Signia by Hilton Atlanta 12/31/24 \u2013 1/1/25 ... December 31. Tuesday. Atlanta\u2019s New Year\u2019s Eve Peach Drop ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events Calendar 2024/2025 | Atlanta, GA","url":"https://atlanta-ga.events/","displayUrl":"https://atlanta-ga.events","snippet":"This calendar is your ultimate guide to the best things to do in Atlanta, GA: ... Dec 31, 2024 3:30 AM . Notre Dame Fighting Irish . Events: 3. McCamish Pavilion . Dec 31, 2024 2:30 PM . Georgia Tech Yellow Jackets Basketball ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.ytkv04WKv8RTyYo4_DsP4gHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"20 free things to do in metro Atlanta in December","url":"https://www.accessatlanta.com/atlanta-things-to-do/20-free-things-to-do-in-metro-atlanta-in-december/RUFTV7LQ2JEGZGUNGNR6FHEARM/","displayUrl":"https://www.accessatlanta.com/atlanta-things-to-do/20-free-things-to-do-in-metro...","snippet":"Here are 20 of the best things to do in Atlanta in December that are totally free. ... and nightly display until Tuesday, Dec. 31; Time: 4 - 8 ... 14, 21, 23, 27, 29 and 31) Holiday Spirit 2024 ..."},{"name":"Cool Things To Do in December in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-december-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-december-in-atlanta","snippet":"Cool Things To Do in December in Atlanta. By Mary Welch. Published November 14, 2023 Last Updated December 2, 2024. December is a great time to be in Atlanta. The list of cool things to do seems endless. There\u2019s unparalleled shopping, great themed entertainment, holiday light displays galore and wonderful restaurants where you can enjoy a ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.B7WMsn5icdXhjRnlj2WcZAHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]}}
```

Here are some events and activities happening in Atlanta on December 31, 2024:

1. **Georgia Tech Yellow Jackets Basketball**  
   - **Time:** 2:30 PM  
   - **Location:** McCamish Pavilion  
   - [More details on Atlanta Events](https://atlanta-ga.events/december/)

2. **New Year’s Eve Celebration at the Georgia Aquarium**  
   - Enjoy DJ and dancing, food, and after-hours access to the aquatic exhibits.  
   - [Details on Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-december.html)

3. **HUGEL New Year's Eve Party**  
   - **Time:** 10:00 PM  
   - **Location:** District Atlanta  
   - [Eventbrite listing](https://www.eventbrite.com/d/ga--atlanta/december-31/)

4. **New Year’s Eve Dinner at Signia by Hilton Atlanta**  
   - Enjoy a special dinner to welcome the new year.  
   - [Discover Atlanta Events](https://discoveratlanta.com/events/main/)

5. **Atlanta Symphony Orchestra: New Year’s Eve Concert**  
   - Enjoy a beautiful musical performance to ring in the new year.  
   - [AllEvents.in listing](https://allevents.in/atlanta/december)

6. **Atlanta’s New Year’s Eve Peach Drop**  
   - Celebrate the new year with Atlanta's iconic Peach Drop event.  
   - [Discover Atlanta Events](https://discoveratlanta.com/events/main/)

![Atlanta Events](https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh&w=80&h=80&c=1&pid=5.1)

These events offer a variety of experiences from sports to music and festive celebrations, making for an exciting New Year's Eve in Atlanta!


In [5]:
output.LastMessage.DisplayAs("text/markdown");

Here are some events and activities happening in Atlanta on December 31, 2024:

1. **Georgia Tech Yellow Jackets Basketball**  
   - **Time:** 2:30 PM  
   - **Location:** McCamish Pavilion  
   - [More details on Atlanta Events](https://atlanta-ga.events/december/)

2. **New Year’s Eve Celebration at the Georgia Aquarium**  
   - Enjoy DJ and dancing, food, and after-hours access to the aquatic exhibits.  
   - [Details on Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-december.html)

3. **HUGEL New Year's Eve Party**  
   - **Time:** 10:00 PM  
   - **Location:** District Atlanta  
   - [Eventbrite listing](https://www.eventbrite.com/d/ga--atlanta/december-31/)

4. **New Year’s Eve Dinner at Signia by Hilton Atlanta**  
   - Enjoy a special dinner to welcome the new year.  
   - [Discover Atlanta Events](https://discoveratlanta.com/events/main/)

5. **Atlanta Symphony Orchestra: New Year’s Eve Concert**  
   - Enjoy a beautiful musical performance to ring in the new year.  
   - [AllEvents.in listing](https://allevents.in/atlanta/december)

6. **Atlanta’s New Year’s Eve Peach Drop**  
   - Celebrate the new year with Atlanta's iconic Peach Drop event.  
   - [Discover Atlanta Events](https://discoveratlanta.com/events/main/)

![Atlanta Events](https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh&w=80&h=80&c=1&pid=5.1)

These events offer a variety of experiences from sports to music and festive celebrations, making for an exciting New Year's Eve in Atlanta!

In [6]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "Web Search and Scrape",
    Instructions = "Use python to get todays date and then use it to find things to do in Atlanta for each of the next four weekends. Be careful to not include events in the past in your final answer. You should perform four searches at once.",
};
var output2 = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output2.Dialog.DisplayAs("text/markdown");


User: Use python to get todays date and then use it to find things to do in Atlanta for each of the next four weekends. Be careful to not include events in the past in your final answer. You should perform four searches at once.

Assistant: I ran this code:
```
from datetime import datetime, timedelta

# Get today's date
today = datetime.now()

# Calculate the dates for the next four weekends (Saturday and Sunday)
weekends = []
for i in range(1, 5):
    saturday = today + timedelta(days=(5 - today.weekday()) + 7 * (i - 1))
    sunday = saturday + timedelta(days=1)
    weekends.append((saturday.strftime('%Y-%m-%d'), sunday.strftime('%Y-%m-%d')))

weekends
```

I called the tool named crawl with these arguments:
```
{"q": "Things to do in Atlanta January 4-5, 2025", "count": 10, "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"Things to do in Atlanta January 4-5, 2025"},"webPages":{"value":[{"name":"Atlanta Events January 2025: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/january/","displayUrl":"https://atlanta-ga.events/january","snippet":"Explore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for January 2025. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month."},{"name":"Atlanta Events \u2013 January 2025 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","snippet":"Events From January 1 To The Weekend Of January 4-5 2025. \u2192 See December events. \u2192 For concerts this week, see concerts in Atlanta in January \u2013 IllumiNights, November 14 2024 - January 19 2025 ... \u2013 Atlanta Boat Show, January 9-12 2025 Annual boating trade show at the Georgia World Congress Center. More details \u2013 Scott Antique Market ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"8 Things to Do in Atlanta This Weekend: Jan. 3-5, 2025 - Time Out","url":"https://www.timeout.com/atlanta/things-to-do/best-things-to-do-in-atlanta-this-weekend","displayUrl":"https://www.timeout.com/atlanta/things-to-do/best-things-to-do-in-atlanta-this-weekend","snippet":"Jan. 4 at Cobb Energy Centre. Get ready to loop through memory lane at Sonic Symphony, a live concert celebrating over 30 years of iconic Sonic the Hedgehog music. From retro 8-bit classics to ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.wA_rMsgTDiYoT68iqJWOpgHaFj\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Discover 2025 Events \u0026amp; Activities in Atlanta, GA | Eventbrite","url":"https://www.eventbrite.com/d/ga--atlanta/2025/","displayUrl":"https://www.eventbrite.com/d/ga--atlanta/2025","snippet":"Save this event: From Day One: Atlanta 2025 Share this event: From Day One: Atlanta 2025. From Day One: Atlanta 2025. Tue, Jan 28, 8:30 AM. Georgia Aquarium. ... NuNu\u0026#39;s Disco Birthday Brunch Jan 5th 2025 Share this event: NuNu\u0026#39;s Disco Birthday Brunch Jan 5th 2025. NuNu\u0026#39;s Disco Birthday Brunch Jan 5th 2025. Sun, Jan 5, 12:00 PM."},{"name":"Events \u0026amp; Things To Do in Atlanta in 2025 - AllEvents.in","url":"https://allevents.in/atlanta/january","displayUrl":"https://allevents.in/atlanta/january","snippet":"Variety Playhouse, Atlanta: 18 Jan 2025: Fulton Lee at The Loft at Center Stage: The Loft at Center Stage, Atlanta: 23 Jan 2025: Kathleen Edwards at City Winery - Atlanta: City Winery - Atlanta: 27 Jan 2025: MJ Lenderman at Variety Playhouse: Variety Playhouse, Atlanta: 28 Jan 2025: Life Love and Beyond Podcast at City Winery - Atlanta: City ..."},{"name":"Atlanta Events Calendar 2024/2025 | Atlanta, GA","url":"https://atlanta-ga.events/","displayUrl":"https://atlanta-ga.events","snippet":"Atlanta Events 2024/2025. Discover the calendar of all current and upcoming events in Atlanta, GA. This calendar is your ultimate guide to the best things to do in Atlanta, GA: ... Jan 4, 2025. Category. Search Events Calendar. Jan Feb Mar Apr May Jun Jul Aug Sep ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.ytkv04WKv8RTyYo4_DsP4gHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events | Explore Georgia","url":"https://exploregeorgia.org/atlanta/events","displayUrl":"https://exploregeorgia.org/atlanta/events","snippet":"Browse the ideas and Atlanta events below to help plan your trip. Find Events. Arts \u0026amp; Culture (21) Culinary, Food, Wine \u0026amp; Beer (12) Entertainment (67) Fairs \u0026amp; Festivals (21) ... January 4, 2025 - May 24, 2025 Painting Zoo Painters. Zoo Atlanta. View all dates. Atlanta. Sunday, January 5. January 5, 2025 - April 27, 2025","thumbnailUrl":"https://www.bing.com/th?id=OIP.qcVhz7_S17NbT6s7cPa8nQHaEi\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"20 Absolutely Fantastic Things To Do In Atlanta This January","url":"https://secretatlanta.co/things-to-do-january/","displayUrl":"https://secretatlanta.co/things-to-do-january","snippet":"Oh, and it\u2019s a wonderful time to try out some brand new things too! If you\u2019re looking for some fun things to do around Atlanta this January, don\u2019t worry, we\u2019ve got ya covered! Keep scrolling to see all the best things to do this January in Atlanta! 1. Center yourself at Oasis at the High Museum of Art High Museum of Art","thumbnailUrl":"https://www.bing.com/th?id=OIP.5lOsdA8tsTOgx5qTd465YAAAAA\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Cool Things To Do in January in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta","snippet":"Cool Things To Do in February in Atlanta; Cool Things To Do in December in Atlanta; Cool Things to Do in March in Atlanta; Family-Friendly Things To Do. Grab the whole family to explore the city, all while trying to keep warm. Delta Flight Museum. 2025 marks the 100th anniversary of Delta Air Lines.","thumbnailUrl":"https://www.bing.com/th?id=OIP.aU9EcatRnz4MDkIKyYbkQgHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta in January | Concerts, Festivals, Tickets \u0026amp; Tour Dates - Songkick","url":"https://www.songkick.com/metro-areas/4120-us-atlanta/january-2025","displayUrl":"https://www.songkick.com/metro-areas/4120-us-atlanta/january-2025","snippet":"Concerts in Atlanta in January. Find tickets to all live music, concerts, tour dates and festivals in and around Atlanta. Currently there are 64 upcoming events. Filter by artist. ... Thursday 02 January 2025; Joe Alterman. Eddie\u0026#39;s Attic, Decatur, GA, US . Friday 03 January 2025; Raheem DeVaughn."}]},"images":{"value":[{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.W-gOzzcskaYfPXC3whhXOgHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/best-things-to-do-in-Atlanta-GA-1.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.5jERbhhnl8LiECUQySoRYAHaG1\u0026pid=Api","contentUrl":"https://uphomes.com/storage/blogposts/February2023/medium/RKuDJcqErH176HD3LyWV.png","hostPageUrl":"https://uphomes.com/blog/things-to-do-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.fqwDUYiMaHe0id6KtJYyfwAAAA\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2022/12/Visiting-Atlanta-in-January-Ultimate-Guide-1.png","hostPageUrl":"https://unexpectedatlanta.com/atlanta-in-january/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.MdoK5WF-Bvvg72JvIonreQHaEG\u0026pid=Api","contentUrl":"https://stonehurstplace.com/wp-content/uploads/2020/01/Things-to-do-in-January-in-Atlanta-1100x609-1.png","hostPageUrl":"https://stonehurstplace.com/our-diary/attractions/things-to-do-in-atlanta-in-january/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.uXSLRLfe7U8MxUYx6YGGVAHaE7\u0026pid=Api","contentUrl":"https://quartzmountain.org/images/resources/things-to-do-atlanta-january_20230726190854.webp","hostPageUrl":"https://quartzmountain.org/article/things-to-do-atlanta-january"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.unmT01rPD7si8w-n3kzOXgHaHa\u0026pid=Api","contentUrl":"https://cdn2.storyasset.link/5f018ed1-7c8d-49e0-9d76-582219d18631/Untitled%20design%20-1-ms-rtheoozqlb.jpg","hostPageUrl":"https://tripononline.com/web-stories/ultimate-things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.Fe-J-AR_X-7c4ZeUHVaHwQAAAA\u0026pid=Api","contentUrl":"https://www.acaponeconnection.com/wp-content/uploads/2022/05/All-the-Fun-Things-to-Do-in-Atlanta-Georgia-1.jpg","hostPageUrl":"https://www.acaponeconnection.com/all-the-fun-things-to-do-in-atlanta-georgia/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.ENao3rD4rUbnHGA6QJHXYgAAAA\u0026pid=Api","contentUrl":"https://res.cloudinary.com/hz3gmuqw6/image/upload/c_fill,q_auto,w_750/f_auto/things-to-do-in-atlanta-phpWlZ13b","hostPageUrl":"https://www.cozymeal.com/magazine/things-to-do-in-atlanta"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.BawFpewdQrecAtgiS9AiOAHaHa\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2023/04/The-Best-Things-to-Do-in-Midtown-Atlanta-500x500.png","hostPageUrl":"https://unexpectedatlanta.com/best-things-to-do-midtown-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.JIWcXtQtpq7A0DXTjwLQiwHaLG\u0026pid=Api","contentUrl":"https://i.pinimg.com/736x/fc/4a/d2/fc4ad281802906d1cfde53017a3e4dcf.jpg","hostPageUrl":"https://www.pinterest.com/pin/744079169673803460/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.VOzVfWJV7_6joNkGhLP7AAHaLH\u0026pid=Api","contentUrl":"https://i.pinimg.com/originals/ce/c5/25/cec525ba57be0022de5d70d1a1f3dd88.jpg","hostPageUrl":"https://www.artofit.org/image-gallery/196891814951760239/18-cool-things-to-do-in-atlanta-georgia-on-your-first-visit/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.B9dwp_jKTmtE_Fruodr10QHaE8\u0026pid=Api","contentUrl":"https://www.treksplorer.com/wp-content/uploads/things-to-do-in-atlanta.jpg","hostPageUrl":"https://www.treksplorer.com/things-to-do-in-atlanta-ga-usa/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.VX9GFXDDPaUhlk5Ajz2jFAHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/best-things-to-do-in-Atlanta-GA-1-2.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.U9xvQMbjg9T0YiriayftJAHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/Things-To-Do-In-Atlanta-pin1.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.OD8mHt8EuAz_-Rm6pRiOFwHaEK\u0026pid=Api","contentUrl":"https://i.ytimg.com/vi/2ufhW2kEYro/maxresdefault.jpg","hostPageUrl":"https://www.youtube.com/watch?v=2ufhW2kEYro"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.871ZuXzoIw6hj9MiYEOM8AHaDt\u0026pid=Api","contentUrl":"https://www.familyvacationcritic.com/wp-content/uploads/sites/19/2019/11/atlanta-centennial-park-hero-1280x640.jpg","hostPageUrl":"https://www.familyvacationcritic.com/free-things-to-do-in-atlanta-with-kids/art/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.cZjlW5ADhS43Nu3fkMZ64gAAAA\u0026pid=Api","contentUrl":"https://a.cdn-hotels.com/gdcs/production90/d1296/c9d18f82-dd9e-4e74-bd3f-5a7fb1d76d99.jpg","hostPageUrl":"https://www.hotels.com/go/usa/things-to-do-atlanta"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.FFnn7XEC9iNqbbTqDM01LQHaD3\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/things-to-do-in-Atlanta-803x420.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.HX79HLcI16lMiECbxV7UWQHaE7\u0026pid=Api","contentUrl":"https://a.cdn-hotels.com/gdcs/production44/d1147/0769c6eb-4e78-45d3-bcca-d80c6e3ac960.jpg?impolicy=fcrop\u0026w=800\u0026h=533\u0026q=medium","hostPageUrl":"https://www.hotels.com/go/usa/things-to-do-atlanta"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.IqznTH1ocAOkn-XVKO4tyQHaL5\u0026pid=Api","contentUrl":"https://i.pinimg.com/originals/f5/05/8e/f5058e44214459ab6506002aee8aa99f.png","hostPageUrl":"https://www.pinterest.at/pin/101-best-things-to-do-in-atlanta-georgia-the-ultimate-attractions-guide--347621664991790215/"}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "Things to do in Atlanta January 11-12, 2025", "count": 10, "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"Things to do in Atlanta January 11-12, 2025"},"webPages":{"value":[{"name":"Atlanta Events \u2013 January 2025 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","snippet":"Things to do in Atlanta, GA in January: calendar of upcoming festivals, Martin Luther King Day events, comedy, musicals, races, conventions, and other events. Go South! ... January 11-12 2025 Annual travel and tourism trade show at the Georgia International Convention Center. More details \u2013 Love\u2019s Labour\u2019s Lost, January 11-26 2025 ..."},{"name":"Atlanta Events January 2025: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/january/","displayUrl":"https://atlanta-ga.events/january","snippet":"Explore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for January 2025. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month.","thumbnailUrl":"https://www.bing.com/th?id=OIP.ytkv04WKv8RTyYo4_DsP4gHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"January 11 events in Atlanta, GA - Eventbrite","url":"https://www.eventbrite.com/d/ga--atlanta/january-11/","displayUrl":"https://www.eventbrite.com/d/ga--atlanta/january-11","snippet":"Save this event: 2025 Atlanta Travel \u0026amp; Adventure Show Share this event: ... Sat, Jan 11, 12:00 PM. The Golden Door Event Hall. Share this event: Powerful Dialogue: ... Sat, Jan 11, 4:00 PM. Atlanta\u0026#39;s Original Uptown Comedy Corner (225) Share this event: ..."},{"name":"Atlanta January 2025: Events, Concerts, Clubs \u0026amp; Things to Do - Experism","url":"https://www.experism.com/atlanta/january","displayUrl":"https://www.experism.com/atlanta/january","snippet":"Things to do in Atlanta January Atlanta Nightclub Events in January ... Jan 9-12, 2025 Georgia World Congress Center Hall C View on Map. The Atlanta Boat Show is an annual event that showcases the latest and greatest in boating technology and accessories. ... Jan 11, 2025 Atlanta View on Map. Don\u0026#39;t miss two-time Emmy Award-winning comedian ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.WKXWyOtbF2Q-oMmg6Y-HiQHaE8\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Discover 2025 Events \u0026amp; Activities in Atlanta, GA | Eventbrite","url":"https://www.eventbrite.com/d/ga--atlanta/2025/","displayUrl":"https://www.eventbrite.com/d/ga--atlanta/2025","snippet":"Vision Board 2025. Sun, Jan 12, 12:00 PM. 3107 Peachtree Rd NE a1. Share this event: ... 2025 Atlanta Travel \u0026amp; Adventure Show Share this event: ... Atlanta Travel \u0026amp; Adventure Show Save this event: 2025 Atlanta Travel \u0026amp; Adventure Show. ELEVATE : IGNITE CONNECTIONS FOR 2025. Sat, Jan 11, 5:00 PM. The Ritz-Carlton, Atlanta. Save this event ..."},{"name":"20 Absolutely Fantastic Things To Do In Atlanta This January","url":"https://secretatlanta.co/things-to-do-january/","displayUrl":"https://secretatlanta.co/things-to-do-january","snippet":"If you\u2019re looking for some fun things to do around Atlanta this January, don\u2019t worry, we\u2019ve got ya covered! ... 12. Check out one of Atlanta\u2019s lesser-known museums ... Where: Woodruff Park | 91 Peachtree Street, Atlanta, GA. When: Until January 12th, 2025. If you\u2019re searching for a dazzling display to check out this January, don\u2019t ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.5lOsdA8tsTOgx5qTd465YAAAAA\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Cool Things To Do in January in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta","snippet":"Find cool things to do in Atlanta during the month of January, from annual festivals to seasonal activities, theatrical performances, and more! ... Coucheron \u002B The Four Seasons, Jan. 10-12. Joe Gatto: LET\u2019S GET INTO IT, Jan. 11. Rachmaninoff \u002B Stravinsky, Jan. 16-18. Music for the Very Young \u2013 The Lion and the Mouse: A Musical Tale for ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.aU9EcatRnz4MDkIKyYbkQgHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Fun, Free Activities for Kids in January 2025: 15 Free Things To Do in ...","url":"https://www.msn.com/en-us/sports/other/fun-free-activities-for-kids-in-january-2025-15-free-things-to-do-in-atlanta-with-kids/ar-AA1mOEey","displayUrl":"https://www.msn.com/en-us/sports/other/fun-free-activities-for-kids-in-january-2025-15...","snippet":"Free Things to Do with Kids in Atlanta January 2025. Chick-fil-A Peach Bowl Fan Fest Downtown. Wednesday, January 1, 2025. ... Sunday, January 12, 2025. On the second Sunday of each month ..."},{"name":"Atlanta in January | Concerts, Festivals, Tickets \u0026amp; Tour Dates - Songkick","url":"https://www.songkick.com/metro-areas/4120-us-atlanta/january-2025","displayUrl":"https://www.songkick.com/metro-areas/4120-us-atlanta/january-2025","snippet":"Concerts in Atlanta in January Find tickets to all live music, concerts, tour dates and festivals in and around Atlanta. Currently there are 64 upcoming events."},{"name":"Atlanta Events | Explore Georgia","url":"https://exploregeorgia.org/atlanta/events","displayUrl":"https://exploregeorgia.org/atlanta/events","snippet":"Browse the ideas and Atlanta events below to help plan your trip. Find Events. Arts \u0026amp; Culture (21) Culinary, Food, Wine \u0026amp; Beer (12) Entertainment (67) Fairs \u0026amp; Festivals (21) ... January 9, 2025 - January 12, 2025 Atlanta Boat Show. Georgia World Congress Center - Hall C. View all dates. Atlanta. Wednesday, February 12. February 12, 2025 - May ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.qcVhz7_S17NbT6s7cPa8nQHaEi\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "Things to do in Atlanta January 18-19, 2025", "count": 10, "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"Things to do in Atlanta January 18-19, 2025"},"webPages":{"value":[{"name":"Atlanta Events January 2025: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/january/","displayUrl":"https://atlanta-ga.events/january","snippet":"Explore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for January 2025. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month.","thumbnailUrl":"https://www.bing.com/th?id=OIP.ytkv04WKv8RTyYo4_DsP4gHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events \u2013 January 2025 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","snippet":"Things to do in Atlanta, GA in January: calendar of upcoming festivals, Martin Luther King Day events, comedy, musicals, races, conventions, and other events. ... \u2013 IllumiNights, November 14 2024 - January 19 2025 Annual after-hours walk-through holiday lights show at Zoo Atlanta, with a mile of illuminated animal lanterns. ... January 18-20 ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Events \u0026amp; Things To Do in Atlanta in 2025 - AllEvents.in","url":"https://allevents.in/atlanta/january","displayUrl":"https://allevents.in/atlanta/january","snippet":"Find events and things to do in January 2025 in Atlanta. Discover parties, concerts, meets, shows, sports, club, reunion, Performance happening in January 2024 in Atlanta. Open Menu. ... Atlanta: 18 Jan 2025: Electric Avenue at Variety Playhouse: Variety Playhouse, Atlanta: 18 Jan 2025: Fulton Lee at The Loft at Center Stage:"},{"name":"Cool Things To Do in January in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta","snippet":"Find cool things to do in Atlanta during the month of January, from annual festivals to seasonal activities, theatrical performances, and more! ... Music for the Very Young \u2013 The Lion and the Mouse: A Musical Tale for Little Ears, Jan. 18-19. SWAG Gospel, Jan. 19. Beethoven Project: Symphony No. 1 and Symphony No. 3, Jan. 23-25. IN UNISON ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.aU9EcatRnz4MDkIKyYbkQgHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Fun, Free Activities for Kids in January 2025: 15 Free Things To Do in ...","url":"https://www.msn.com/en-us/sports/other/fun-free-activities-for-kids-in-january-2025-15-free-things-to-do-in-atlanta-with-kids/ar-AA1mOEey","displayUrl":"https://www.msn.com/en-us/sports/other/fun-free-activities-for-kids-in-january-2025-15...","snippet":"Free Things to Do with Kids in Atlanta January 2025. Chick-fil-A Peach Bowl Fan Fest Downtown. ... January 18, 2025. Mini-DIYers head to Lowe\u2019s to attend a workshop hosted by a Lowes employee ..."},{"name":"20 free things to do in metro Atlanta in January","url":"https://www.accessatlanta.com/atlanta-things-to-do/20-free-things-to-do-in-metro-atlanta-in-january/Q3EOZY6OHBE2HLT4QN5ZIBLBQU/","displayUrl":"https://www.accessatlanta.com/atlanta-things-to-do/20-free-things-to-do-in-metro...","snippet":"Here are 20 of the best things to do in Atlanta in January that are ... Kick off 2025 in style with Yoga \u0026amp; Sound at The ... Website: atlantahistorycenter.com; Date: Saturday, Jan 18; Time: 10 a.m ..."},{"name":"Top Attractions to Visit this Winter in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/attractions-to-visit-this-winter-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/attractions-to-visit-this-winter-in...","snippet":"Children\u2019s Museum of Atlanta offers families an opportunity to come together with a new exhibit from Jan. 25, 2025, through May 4, 2025. \u201CXOXO: An Exhibit About Love \u0026amp; Forgiveness\u201D is designed to foster conversation and connection and is designed for guests of all ages. Where: 275 Centennial Olympic Park Drive NW","thumbnailUrl":"https://www.bing.com/th?id=OIP.z8MiD8hlIeihw0QjVGJzrgHaE7\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta in January | Concerts, Festivals, Tickets \u0026amp; Tour Dates - Songkick","url":"https://www.songkick.com/metro-areas/4120-us-atlanta/january-2025","displayUrl":"https://www.songkick.com/metro-areas/4120-us-atlanta/january-2025","snippet":"Concerts in Atlanta in January. Find tickets to all live music, concerts, tour dates and festivals in and around Atlanta. ... Saturday 18 January 2025; Fulton Lee. The Loft, Atlanta, GA, US . Bit Brigade. ... Sat 19 Oct 2024 Boot Barn Hall at Bourbon Brothers Gainesville, GA, US. Rare Americans. Sat 15 ..."},{"name":"20 Absolutely Fantastic Things To Do In Atlanta This January","url":"https://secretatlanta.co/things-to-do-january/","displayUrl":"https://secretatlanta.co/things-to-do-january","snippet":"If you\u2019re looking for some fun things to do around Atlanta this January, don\u2019t worry, we\u2019ve got ya covered! ... Woodruff Park | 91 Peachtree Street, Atlanta, GA. When: Until January 12th, 2025. If you\u2019re searching for a dazzling display to check out this January, don\u2019t worry, we\u2019ve got you covered! ... 18. Set sail on an emotional ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.5lOsdA8tsTOgx5qTd465YAAAAA\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta January 2025: Events, Concerts, Clubs \u0026amp; Things to Do - Experism","url":"https://www.experism.com/atlanta/january","displayUrl":"https://www.experism.com/atlanta/january","snippet":"Don\u2019t miss the Atlanta\u0026#39;s 2025 Chick-fil-A Peach Bowl on January 1. As one of college football\u2019s premier postseason games, the Peach Bowl brings together two of the nation\u2019s top teams for a gridiron showdown at Mercedes-Benz Stadium. The Peach Bowl experience includes pre-game festivities, live entertainment, and interactive fan zones.","thumbnailUrl":"https://www.bing.com/th?id=OIP.WKXWyOtbF2Q-oMmg6Y-HiQHaE8\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]},"images":{"value":[{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.W-gOzzcskaYfPXC3whhXOgHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/best-things-to-do-in-Atlanta-GA-1.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.5jERbhhnl8LiECUQySoRYAHaG1\u0026pid=Api","contentUrl":"https://uphomes.com/storage/blogposts/February2023/medium/RKuDJcqErH176HD3LyWV.png","hostPageUrl":"https://uphomes.com/blog/things-to-do-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.fqwDUYiMaHe0id6KtJYyfwAAAA\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2022/12/Visiting-Atlanta-in-January-Ultimate-Guide-1.png","hostPageUrl":"https://unexpectedatlanta.com/atlanta-in-january/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.MdoK5WF-Bvvg72JvIonreQHaEG\u0026pid=Api","contentUrl":"https://stonehurstplace.com/wp-content/uploads/2020/01/Things-to-do-in-January-in-Atlanta-1100x609-1.png","hostPageUrl":"https://stonehurstplace.com/our-diary/attractions/things-to-do-in-atlanta-in-january/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.VOzVfWJV7_6joNkGhLP7AAHaLH\u0026pid=Api","contentUrl":"https://i.pinimg.com/originals/ce/c5/25/cec525ba57be0022de5d70d1a1f3dd88.jpg","hostPageUrl":"https://www.artofit.org/image-gallery/196891814951760239/18-cool-things-to-do-in-atlanta-georgia-on-your-first-visit/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.uXSLRLfe7U8MxUYx6YGGVAHaE7\u0026pid=Api","contentUrl":"https://quartzmountain.org/images/resources/things-to-do-atlanta-january_20230726190854.webp","hostPageUrl":"https://quartzmountain.org/article/things-to-do-atlanta-january"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.unmT01rPD7si8w-n3kzOXgHaHa\u0026pid=Api","contentUrl":"https://cdn2.storyasset.link/5f018ed1-7c8d-49e0-9d76-582219d18631/Untitled%20design%20-1-ms-rtheoozqlb.jpg","hostPageUrl":"https://tripononline.com/web-stories/ultimate-things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.Fe-J-AR_X-7c4ZeUHVaHwQAAAA\u0026pid=Api","contentUrl":"https://www.acaponeconnection.com/wp-content/uploads/2022/05/All-the-Fun-Things-to-Do-in-Atlanta-Georgia-1.jpg","hostPageUrl":"https://www.acaponeconnection.com/all-the-fun-things-to-do-in-atlanta-georgia/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.JIWcXtQtpq7A0DXTjwLQiwHaLG\u0026pid=Api","contentUrl":"https://i.pinimg.com/736x/fc/4a/d2/fc4ad281802906d1cfde53017a3e4dcf.jpg","hostPageUrl":"https://www.pinterest.com/pin/744079169673803460/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.ENao3rD4rUbnHGA6QJHXYgAAAA\u0026pid=Api","contentUrl":"https://res.cloudinary.com/hz3gmuqw6/image/upload/c_fill,q_auto,w_750/f_auto/things-to-do-in-atlanta-phpWlZ13b","hostPageUrl":"https://www.cozymeal.com/magazine/things-to-do-in-atlanta"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.VX9GFXDDPaUhlk5Ajz2jFAHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/best-things-to-do-in-Atlanta-GA-1-2.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.vjZ_V7Yjzfqyphfsj_SBjgHaEK\u0026pid=Api","contentUrl":"https://daytripqueen.com/wp-content/uploads/2021/07/Free-things-to-do-in-Atlanta.jpg","hostPageUrl":"https://daytripqueen.com/free-things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.eKakKX-7lUL-8AAx78alPQHaHa\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2023/04/The-Best-Things-to-Do-in-Midtown-Atlanta.png","hostPageUrl":"https://unexpectedatlanta.com/best-things-to-do-midtown-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.B9dwp_jKTmtE_Fruodr10QHaE8\u0026pid=Api","contentUrl":"https://www.treksplorer.com/wp-content/uploads/things-to-do-in-atlanta.jpg","hostPageUrl":"https://www.treksplorer.com/things-to-do-in-atlanta-ga-usa/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.FFnn7XEC9iNqbbTqDM01LQHaD3\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/things-to-do-in-Atlanta-803x420.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.U9xvQMbjg9T0YiriayftJAHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/Things-To-Do-In-Atlanta-pin1.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.3DCmfuvb9PKm59kY35gOowHaD5\u0026pid=Api","contentUrl":"https://www.accessatlanta.com/resizer/st3ipUHpjhivT85uXQnam1p9Dk8=/1280x673/smart/cloudfront-us-east-1.images.arcpublishing.com/ajc/5SITMG7WA5BB3FUUWKHED4CF4U.jpg","hostPageUrl":"https://www.accessatlanta.com/atlanta-things-to-do/10-things-to-do-in-atlanta-this-weekend/VNXLGWBIMFBNNJUH5VYLLFZQGA/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.yRC0krj1Jbi8qtYhSLLimgHaE8\u0026pid=Api","contentUrl":"https://lp-cms-production.imgix.net/2021-05/GettyRF_486068945.jpg?auto=compress\u0026fit=crop\u0026fm=auto\u0026sharp=10\u0026vib=20\u0026w=1200\u0026h=800","hostPageUrl":"https://www.lonelyplanet.com/articles/top-things-to-do-in-atlanta"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.IqznTH1ocAOkn-XVKO4tyQHaL5\u0026pid=Api","contentUrl":"https://i.pinimg.com/originals/f5/05/8e/f5058e44214459ab6506002aee8aa99f.png","hostPageUrl":"https://www.pinterest.at/pin/101-best-things-to-do-in-atlanta-georgia-the-ultimate-attractions-guide--347621664991790215/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.-1BVEwbN81Oc9jxAOaX2SQHaE5\u0026pid=Api","contentUrl":"https://southerntrippers.com/wp-content/uploads/2022/08/things-to-do-in-atlanta-at-night-centennial-olympic-park-.jpg","hostPageUrl":"https://southerntrippers.com/things-to-do-in-atlanta-at-night/"}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "Things to do in Atlanta January 25-26, 2025", "count": 10, "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"Things to do in Atlanta January 25-26, 2025"},"webPages":{"value":[{"name":"Atlanta Events \u2013 January 2025 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","snippet":"Events From January 21 To The Weekend Of January 25-26 2025. \u2192 For concerts this week, see concerts in Atlanta in January \u2013 Love\u2019s Labour\u2019s Lost, January 11-26 2025 Play at the Shakespeare Tavern Playhouse. More details \u2013 Gimme Please!, January 16 - February 16 2025 Children\u2019s play at Alliance Theatre. More details"},{"name":"Atlanta Events January 2025: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/january/","displayUrl":"https://atlanta-ga.events/january","snippet":"Explore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for January 2025. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month.","thumbnailUrl":"https://www.bing.com/th?id=OIP.ytkv04WKv8RTyYo4_DsP4gHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"25 Reasons to Visit Atlanta in 2025","url":"https://discoveratlanta.com/stories/things-to-do/25-reasons-to-visit-atlanta-in-2025/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/25-reasons-to-visit-atlanta-in-2025","snippet":"The Atlanta Chinese Dance Company performs on stage on Jan. 25,2025, at Aurora Children\u2019s Playhouse. Here are more ways to celebrate Chinese New Year in Atlanta. Spring Festivals. Atlanta Film Festival 2025 takes place April 24 through May 4, 2025, at the Plaza Theatre \u0026amp; Tara Theatre."},{"name":"Cool Things To Do in January in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta","snippet":"Moon Zooz: Too Many Zooz \u002B Moon Hooch with Honeycomb, Jan. 25. MJ Lenderman \u0026amp; The Wind, Jan. 26-27. James McMurtry, Jan. 29. Feeling Sporty? ATLANTA, GA \u2013 JANUARY 26: Trae Young #11 of the Atlanta Hawks drives to the basket against the LA Clippers on January 26, 2021 at State Farm Arena in Atlanta, Georgia.","thumbnailUrl":"https://www.bing.com/th?id=OIP.aU9EcatRnz4MDkIKyYbkQgHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Events \u0026amp; Things To Do in Atlanta in 2025 - AllEvents.in","url":"https://allevents.in/atlanta/january","displayUrl":"https://allevents.in/atlanta/january","snippet":"Find events and things to do in January 2025 in Atlanta. Discover parties, concerts, meets, shows, sports, club, reunion, Performance happening in January 2024 in Atlanta. ... Sat, 25 Jan Trending events in january in Atlanta. Featured. NYE in the ATL 2025 : New Year\u0026#39;s Eve Atlanta Crowne Plaza Atlanta Perimeter at Ravinia 31 Dec-01 Jan ..."},{"name":"Atlanta January 2025: Events, Concerts, Clubs \u0026amp; Things to Do - Experism","url":"https://www.experism.com/atlanta/january","displayUrl":"https://www.experism.com/atlanta/january","snippet":"Don\u2019t miss the Atlanta\u0026#39;s 2025 Chick-fil-A Peach Bowl on January 1. As one of college football\u2019s premier postseason games, the Peach Bowl brings together two of the nation\u2019s top teams for a gridiron showdown at Mercedes-Benz Stadium. The Peach Bowl experience includes pre-game festivities, live entertainment, and interactive fan zones.","thumbnailUrl":"https://www.bing.com/th?id=OIP.WKXWyOtbF2Q-oMmg6Y-HiQHaE8\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Concerts in January 2025 - Eventworld","url":"https://www.eventworld.co/atlanta/january/","displayUrl":"https://www.eventworld.co/atlanta/january","snippet":"Find upcoming concerts, gigs \u0026amp; tour dates in Atlanta, GA in january 2025. ... January 26, 2025: Cobb Energy Performing Arts Centre : $50. Buy tickets. Concerts. Sonic Symphony ... January 25, 2025: Robert Ferst Center for the Arts - Georgia Tech : $90. Buy tickets. Concerts. Silly Goose ..."},{"name":"20\u002B Not-To-Miss January Events - Atlanta Parent","url":"https://www.atlantaparent.com/january-events/","displayUrl":"https://www.atlantaparent.com/january-events","snippet":"Jan. 18-March 2. Speedway Open House at Atlanta Motor Speedway. See the 1.54-mile track, enjoy games, interact with special guests and more. Jan. 25. Trains, Trains, Trains at The Southern Museum. View an enormous model train display, and enjoy train-themed activities and historic trains. Jan. 25-26. Fairy House Breakfast at Chattahoochee ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.-jkVHRYVqNJ00Eq6HFDgIQAAAA\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta in January | Concerts, Festivals, Tickets \u0026amp; Tour Dates - Songkick","url":"https://www.songkick.com/metro-areas/4120-us-atlanta/january-2025","displayUrl":"https://www.songkick.com/metro-areas/4120-us-atlanta/january-2025","snippet":"Concerts in Atlanta in January. Find tickets to all live music, concerts, tour dates and festivals in and around Atlanta. Currently there are 64 upcoming events. Filter by artist. ... Sat 25 Jan 2025 Buckhead Theatre Atlanta, GA, US. Cool Company. Fri 22 Nov 2024 The Masquerade - Purgatory Atlanta, GA, US. pigeon pit. Mon 28 ..."},{"name":"Atlanta Events Calendar 2024/2025 | Atlanta, GA","url":"https://atlanta-ga.events/","displayUrl":"https://atlanta-ga.events","snippet":"Atlanta Events 2024/2025. Discover the calendar of all current and upcoming events in Atlanta, GA. This calendar is your ultimate guide to the best things to do in Atlanta, GA: ... Jan 4, 2025. Category. Search Events Calendar. Jan Feb Mar Apr May Jun Jul Aug Sep ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.ytkv04WKv8RTyYo4_DsP4gHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]},"images":{"value":[{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.W-gOzzcskaYfPXC3whhXOgHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/best-things-to-do-in-Atlanta-GA-1.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.5jERbhhnl8LiECUQySoRYAHaG1\u0026pid=Api","contentUrl":"https://uphomes.com/storage/blogposts/February2023/medium/RKuDJcqErH176HD3LyWV.png","hostPageUrl":"https://uphomes.com/blog/things-to-do-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.fqwDUYiMaHe0id6KtJYyfwAAAA\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2022/12/Visiting-Atlanta-in-January-Ultimate-Guide-1.png","hostPageUrl":"https://unexpectedatlanta.com/atlanta-in-january/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.AsEhWz77ez1CIx9RVZsvFAHaEh\u0026pid=Api","contentUrl":"https://res.cloudinary.com/atlanta/images/v1636654462/newAtlanta.com/Atlanta_OneMusicFest_COP-1/Atlanta_OneMusicFest_COP-1.jpg?_i=AA","hostPageUrl":"https://martellewgennie.pages.dev/ycqmcw-things-to-do-in-atlanta-this-weekend-events-dgpyjjsmr-photos/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.Fe-J-AR_X-7c4ZeUHVaHwQAAAA\u0026pid=Api","contentUrl":"https://www.acaponeconnection.com/wp-content/uploads/2022/05/All-the-Fun-Things-to-Do-in-Atlanta-Georgia-1.jpg","hostPageUrl":"https://www.acaponeconnection.com/all-the-fun-things-to-do-in-atlanta-georgia/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.SeKsuJEAl7i4y99RBVdFjAHaE7\u0026pid=Api","contentUrl":"https://tds.indianeagle.com/wp-content/uploads/2023/11/Things-To-Do-in-Atlanta-1.png","hostPageUrl":"https://www.thecommongroundnetwork.com/blog/international-travel/discover-best-things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.8B9LmuRXqgsRTEY6n_RNlAHaHa\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2023/06/The-Best-Things-to-Do-in-Atlanta.png","hostPageUrl":"https://unexpectedatlanta.com/atlanta-event-calendar/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.ENao3rD4rUbnHGA6QJHXYgAAAA\u0026pid=Api","contentUrl":"https://res.cloudinary.com/hz3gmuqw6/image/upload/c_fill,q_auto,w_750/f_auto/things-to-do-in-atlanta-phpWlZ13b","hostPageUrl":"https://www.cozymeal.com/magazine/things-to-do-in-atlanta"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.JIWcXtQtpq7A0DXTjwLQiwHaLG\u0026pid=Api","contentUrl":"https://i.pinimg.com/736x/fc/4a/d2/fc4ad281802906d1cfde53017a3e4dcf.jpg","hostPageUrl":"https://www.pinterest.com/pin/744079169673803460/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.BawFpewdQrecAtgiS9AiOAHaHa\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2023/04/The-Best-Things-to-Do-in-Midtown-Atlanta-500x500.png","hostPageUrl":"https://unexpectedatlanta.com/best-things-to-do-midtown-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.uXSLRLfe7U8MxUYx6YGGVAHaE7\u0026pid=Api","contentUrl":"https://quartzmountain.org/images/resources/things-to-do-atlanta-january_20230726190854.webp","hostPageUrl":"https://quartzmountain.org/article/things-to-do-atlanta-january"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.MdoK5WF-Bvvg72JvIonreQHaEG\u0026pid=Api","contentUrl":"https://stonehurstplace.com/wp-content/uploads/2020/01/Things-to-do-in-January-in-Atlanta-1100x609-1.png","hostPageUrl":"https://stonehurstplace.com/our-diary/attractions/things-to-do-in-atlanta-in-january/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.VX9GFXDDPaUhlk5Ajz2jFAHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/best-things-to-do-in-Atlanta-GA-1-2.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.unmT01rPD7si8w-n3kzOXgHaHa\u0026pid=Api","contentUrl":"https://cdn2.storyasset.link/5f018ed1-7c8d-49e0-9d76-582219d18631/Untitled%20design%20-1-ms-rtheoozqlb.jpg","hostPageUrl":"https://tripononline.com/web-stories/ultimate-things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.FFnn7XEC9iNqbbTqDM01LQHaD3\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/things-to-do-in-Atlanta-803x420.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.U9xvQMbjg9T0YiriayftJAHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/Things-To-Do-In-Atlanta-pin1.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.cqJOjMah_PawHVRXufjEtQAAAA\u0026pid=Api","contentUrl":"https://365atlantatraveler.com/wp-content/uploads/2019/03/things-to-do-atlanta-200x300.jpg","hostPageUrl":"https://365atlantatraveler.com/things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.yRC0krj1Jbi8qtYhSLLimgHaE8\u0026pid=Api","contentUrl":"https://lp-cms-production.imgix.net/2021-05/GettyRF_486068945.jpg?auto=compress\u0026fit=crop\u0026fm=auto\u0026sharp=10\u0026vib=20\u0026w=1200\u0026h=800","hostPageUrl":"https://www.lonelyplanet.com/articles/top-things-to-do-in-atlanta"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.v6IPB_NTvbmy-pxyZSCA0QHaE7\u0026pid=Api","contentUrl":"http://rwethereyetmom.com/wp-content/uploads/2015/08/6-Things-to-do-in-Atlanta-1024x681.jpg","hostPageUrl":"http://rwethereyetmom.com/6-things-to-do-in-atlanta.html"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.TNv2qzXsSFvw43t4zu0gGgHaE8\u0026pid=Api","contentUrl":"https://www.travelandleisure.com/thmb/3NRm4q9Qg2bdUqXvenPBVeIloV4=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/atlanta-skyline-TODOATL0122-195afa04632e43f5be27ef2dcc94ca3b.jpg","hostPageUrl":"https://www.travelandleisure.com/attractions/best-things-to-do-in-atlanta-georgia"}]}}
```

Here's a list of events and activities happening in Atlanta over the next four weekends of January 2025:

### January 4-5, 2025
1. **Sonic Symphony** at Cobb Energy Centre - A live concert celebrating over 30 years of Sonic the Hedgehog music. [Time Out](https://www.timeout.com/atlanta/things-to-do/best-things-to-do-in-atlanta-this-weekend)
2. **NuNu's Disco Birthday Brunch** - A fun brunch event happening on January 5th. [Eventbrite](https://www.eventbrite.com/d/ga--atlanta/2025/)
3. **IllumiNights** at Zoo Atlanta - A holiday lights show featuring illuminated animal lanterns. [Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html)

![Sonic Symphony](https://www.busytourist.com/wp-content/uploads/2020/07/best-things-to-do-in-Atlanta-GA-1.jpg)

### January 11-12, 2025
1. **Atlanta Travel & Adventure Show** - An annual travel and tourism trade show at the Georgia International Convention Center. [Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html)
2. **Love’s Labour’s Lost** - A play running from January 11-26 at the Shakespeare Tavern Playhouse. [Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html)
3. **Joe Gatto: LET’S GET INTO IT** - A comedy show on January 11. [Discover Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta/)

![Atlanta Travel & Adventure Show](https://uphomes.com/storage/blogposts/February2023/medium/RKuDJcqErH176HD3LyWV.png)

### January 18-19, 2025
1. **IllumiNights** at Zoo Atlanta continues until January 19th. [Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html)
2. **Electric Avenue** at Variety Playhouse - A concert event on January 18. [AllEvents.in](https://allevents.in/atlanta/january)
3. **Music for the Very Young – The Lion and the Mouse** - A musical tale for little ears on January 18-19. [Discover Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta/)

![Electric Avenue](https://unexpectedatlanta.com/wp-content/uploads/2022/12/Visiting-Atlanta-in-January-Ultimate-Guide-1.png)

### January 25-26, 2025
1. **Atlanta Chinese Dance Company Performance** - Celebrate Chinese New Year on January 25 at Aurora Children’s Playhouse. [Discover Atlanta](https://discoveratlanta.com/stories/things-to-do/25-reasons-to-visit-atlanta-in-2025/)
2. **Trains, Trains, Trains** at The Southern Museum - Enjoy train-themed activities on January 25. [Atlanta Parent](https://www.atlantaparent.com/january-events/)
3. **Moon Zooz: Too Many Zooz + Moon Hooch** - A music event on January 25. [Discover Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta/)

![Atlanta Chinese Dance Company](https://stonehurstplace.com/wp-content/uploads/2020/01/Things-to-do-in-January-in-Atlanta-1100x609-1.png)

These events offer a diverse range of activities from concerts and shows to cultural celebrations, ensuring there's something for everyone to enjoy in Atlanta this January.


In [7]:
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Formatting;

output2.LastMessage.DisplayAs("text/markdown");


Here's a list of events and activities happening in Atlanta over the next four weekends of January 2025:

### January 4-5, 2025
1. **Sonic Symphony** at Cobb Energy Centre - A live concert celebrating over 30 years of Sonic the Hedgehog music. [Time Out](https://www.timeout.com/atlanta/things-to-do/best-things-to-do-in-atlanta-this-weekend)
2. **NuNu's Disco Birthday Brunch** - A fun brunch event happening on January 5th. [Eventbrite](https://www.eventbrite.com/d/ga--atlanta/2025/)
3. **IllumiNights** at Zoo Atlanta - A holiday lights show featuring illuminated animal lanterns. [Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html)

![Sonic Symphony](https://www.busytourist.com/wp-content/uploads/2020/07/best-things-to-do-in-Atlanta-GA-1.jpg)

### January 11-12, 2025
1. **Atlanta Travel & Adventure Show** - An annual travel and tourism trade show at the Georgia International Convention Center. [Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html)
2. **Love’s Labour’s Lost** - A play running from January 11-26 at the Shakespeare Tavern Playhouse. [Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html)
3. **Joe Gatto: LET’S GET INTO IT** - A comedy show on January 11. [Discover Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta/)

![Atlanta Travel & Adventure Show](https://uphomes.com/storage/blogposts/February2023/medium/RKuDJcqErH176HD3LyWV.png)

### January 18-19, 2025
1. **IllumiNights** at Zoo Atlanta continues until January 19th. [Go South Atlanta](https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html)
2. **Electric Avenue** at Variety Playhouse - A concert event on January 18. [AllEvents.in](https://allevents.in/atlanta/january)
3. **Music for the Very Young – The Lion and the Mouse** - A musical tale for little ears on January 18-19. [Discover Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta/)

![Electric Avenue](https://unexpectedatlanta.com/wp-content/uploads/2022/12/Visiting-Atlanta-in-January-Ultimate-Guide-1.png)

### January 25-26, 2025
1. **Atlanta Chinese Dance Company Performance** - Celebrate Chinese New Year on January 25 at Aurora Children’s Playhouse. [Discover Atlanta](https://discoveratlanta.com/stories/things-to-do/25-reasons-to-visit-atlanta-in-2025/)
2. **Trains, Trains, Trains** at The Southern Museum - Enjoy train-themed activities on January 25. [Atlanta Parent](https://www.atlantaparent.com/january-events/)
3. **Moon Zooz: Too Many Zooz + Moon Hooch** - A music event on January 25. [Discover Atlanta](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta/)

![Atlanta Chinese Dance Company](https://stonehurstplace.com/wp-content/uploads/2020/01/Things-to-do-in-January-in-Atlanta-1100x609-1.png)

These events offer a diverse range of activities from concerts and shows to cultural celebrations, ensuring there's something for everyone to enjoy in Atlanta this January.

## Clean Up

In [8]:
var assistant = (await client.AssistantList())?.Data?.FirstOrDefault(o => o.Name == "Web Search and Scrape");
if(assistant != null) {
    await client.AssistantDelete(assistant.Id);
    Console.WriteLine("Deleted assistant");
}
else
{
    Console.WriteLine("Didn't find Web Search and Scrape");
}

Deleted assistant
